In [1]:
reset()
%display latex
Partitions.options.latex="list"
load("setup.sage")

KT_vars = ['x','y','z1','z2','z3','z4','z5','z6','z7','z8','z9',
           'v1','v2','v3','v4','v5','v6','v7','v8','v9']
KT_ring = LaurentPolynomialRing(QQ, KT_vars)

coho_vars = ['t','m','l1','l2','l3','l4','l5','l6','l7','l8','l9',
             'w1','w2','w3','w4','w5','w6','w7','w8','w9']
coho_ring = LaurentPolynomialRing(QQ, coho_vars)


x,y,z1,z2,z3,z4,z5,z6,z7,z8,z9,v1,v2,v3,v4,v5,v6,v7,v8,v9=KT_ring.gens()
t,m,l1,l2,l3,l4,l5,l6,l7,l8,l9,w1,w2,w3,w4,w5,w6,w7,w8,w9= coho_ring.gens()
b=var('b')
'''   w=SR.var('w',5)   '''
v=[v1,v2,v3,v4,v5,v6,v7,v8,v9]
w=[w1,w2,w3,w4,w5,w6,w7,w8,w9]
z=[z1,z2,z3,z4,z5,z6,z7,z8,z9]
L=var('l')


def character(p,x=x,y=y):
    return sum(y^(-r) * x^(-c) for r, c in p.cells())

def character_vb(p, w,x=x,y=y):
    Q = character(p,x,y)
    return sum(Q * x^(w[2*i]) * y^(w[2*i+1]) for i in range(len(w)/2))

def dual(f):
    R = f.parent()
    g= R.zero()
    for exp, coeff in f.dict().items():
        g+= R({exp.emul(-1): coeff})
    return g

def tangent(p,x=x,y=y):
    total=0
    for r,c in p.cells():
        arm=p[r]-c-1
        leg=p.conjugate()[c]-r-1
        total+=y^(-leg)*x^(arm+1)+y^(leg+1)*x^(-arm)
    return total

def canonical_bundle(p,x=x,y=y):
    return character_vb(p, (-1,-1),x,y)


def structure_sheaf(p,x=x,y=y):
    return character_vb(p, (0,0),x,y)

def top_chern(f):
    numer, denom = [], []
    for coeff, mon in list(f):
        wt = from_monomial(mon)
        #if wt==0:
            #continue
        if coeff > 0:
            numer += (coeff) * [wt]
        elif coeff < 0:
            denom += (-coeff) * [wt]
    
    return coho_ring.fraction_field()(prod(numer) / prod(denom))

def determinant(f):
    if f in ZZ:
        return 1
    else:
        return prod((mon)^coeff for coeff, mon in f)

def from_monomial(m):
    return sum(m.degree(KT_ring.gen(i)) * coho_ring.gen(i) for i in range(len(coho_ring.gens())))

def from_character(f):
    numer, denom = [], []

    for coeff, mon in list(f):
        wt = from_monomial(mon)
        if coeff > 0:
            numer += (coeff) * [wt]
        elif coeff < 0:
            denom += (-coeff) * [wt]
    return numer, denom

def measure_unsymmetrized(f, inv=True):
    """
    computes 1/(1-x^-1) for K-thy class x. If inv=false, returns 1/(1-x)
    """
    R = f.parent()
    
    if f.parent() is ZZ:
        L = [(ZZ(f), 1)]
    else:
        L = list(f)

    numer, denom = R.one(), R.one()
    for coeff, monomial in L:
        if inv:
            term = 1 - monomial^-1
        else:
            term = 1 - monomial
        if monomial==1:
            term = 0
        if coeff < 0:
            numer *= term ** (-coeff)
        elif coeff > 0:
            denom *= term ** coeff
    return numer / denom



def degn(f,n,b=b):
    g=f.taylor(b,0,n+2)
    L=g.coefficients(b)
    for i in range(len(L)):
        if L[i][1]==n:
            return L[i][0]
    return 0

    

In [2]:
def partitions(Sum, K):
    '''computes fixed components of Quot by CC^* action.'''
    total=[]
    def ptn(Sum, K, lst, total):
        if K == 0:
            if Sum == 0:
                total+=[lst]
            return
        for i in range(0, min(Sum + 1, Sum + 1)):
            ptn(Sum - i, K - 1, lst + [i],total)
    ptn(Sum,K,[],total)
    return total


def fixed_locus(n, N):
    '''gives list of fixed points of Hilb^n1 x Hilb^n2 x ... x Hilb^nN, input (n1,n2,...,nN)'''
    total=[]
    ptns=partitions(n,N)
    for j in range(len(ptns)):
        ptn=ptns[j]
        lst=[]
        for i in range(N):
            lst+=[Partitions(ptn[i]).list()]
        total+=[p for p in itertools.product(*lst)]
    return total

def Nvir(Z):
    '''virtual normal bundle of quot scheme at (Z_1,...,Z_N)'''
    total=0
    for i in range(len(Z)):
        for j in range(len(Z)):
            total+=(1-character(Z[i],x=x^-1,y=x)*(1-x)*(1-1/x))*character(Z[j],x=x,y=x^-1)*(z[j]/z[i])
    return total+1

def total_chern_quot(Z,rank):
    total=1
    for i in range(len(Z)):
        Q = character(Z[i],x,1/x)*(z[i])
        roots = []
        for coeff, mon in list(Q):
            wt = from_monomial(mon)
            for i in range(rank):
                roots += (coeff) * [wt+w[i]]
        total *= prod((1+r) for r in roots)
    return total

def top_chern_quot(Z):
    total=1
    for i in range(len(Z)):
        Q = character(Z[i],x,1/x)*(z[i])
        roots = []
        for coeff, mon in list(Q):
            wt = from_monomial(mon)
            for i in range(rank):
                roots += (coeff) * [wt+w[i]]
        total *= prod((r) for r in roots)
    return total

def Chern_Num_quot(n,N,rank):
    total=0
    maxcounter=len(fixed_locus(n,N))
    counter=0
    for Z in fixed_locus(n,N):
        add=top_chern(-Nvir(Z))*total_chern_quot(Z,rank)
        total+=add
        counter+=1
        print("added"+str(counter)+"/"+str(maxcounter))
    return total

def DT_quot(n,N):
    total=0
    for Z in fixed_locus(n,N):
        add=top_chern(-Nvir(Z))
        total+=add
    return total

def total_segre_quot(Z,rank):
    total=1
    for i in range(len(Z)):
        Q = character(Z[i],x,1/x)*(z[i])
        roots = []
    
        for coeff, mon in list(Q):
            wt = from_monomial(mon)
            for i in range(rank):
                roots += (coeff) * [wt+w[i]]
        total *= prod(1/(1+r) for r in roots)
    return total

def Segre_Num_quot(n,N,rank):
    total=0
    maxcounter=len(fixed_locus(n,N))
    counter=0
    for Z in fixed_locus(n,N):
        add=top_chern(-Nvir(Z))*total_segre_quot(Z,rank)
        total+=add
        counter+=1
        print("added"+str(counter)+"/"+str(maxcounter))
    return total

def Segre_Num_quot_sym(n,N,rank):
    total=0
    maxcounter=len(fixed_locus(n,N))
    counter=0
    for Z in fixed_locus(n,N):
        add=top_chern(-Nvir(Z))*total_segre_quot(Z,rank)
        total+=add(t=7,w1=15,w2=2,w3=3,l1=17,l2=5,l3=6)
        counter+=1
        print("added"+str(counter)+"/"+str(maxcounter))
    return total


def determinant_quot(Z,n,rank):
    det=1
    for i in range(len(Z)):
        Q = character(Z[i],x,1/x)*(z[i])
        det *= determinant(Q)
    return (det^rank)*((prod(v[i] for i in range(rank)))^n)

def Verlinde_Num_quot(n,N,rank):
    total=0
    maxcounter=len(fixed_locus(n,N))
    counter=0
    for Z in fixed_locus(n,N):
        add=measure_unsymmetrized(Nvir(Z))*\
            determinant_quot(Z,n,rank)
        total+=add
        counter+=1
        print("added"+str(counter)+"/"+str(maxcounter))
    return total

def Verlinde_Num_quot_inv(n,N,rank):
    total=0
    maxcounter=len(fixed_locus(n,N))
    counter=0
    for Z in fixed_locus(n,N):
        add=measure_unsymmetrized(Nvir(Z))*\
            (1/determinant_quot(Z,n,rank))
        total+=add
        counter+=1
        print("added"+str(counter)+"/"+str(maxcounter))
    return total

def todd_class_fast(f,n):
    R = f.parent()
    total=1
    if f.parent() is ZZ:
        L = [(ZZ(f), 1)]
    else:
        L = list(f)
    for coeff, mon in L:
        t=1
        if mon==1:
            term = 0
        if coeff < 0:
            term = 1+sum((-1)^(k+1)*(from_monomial(mon))^(k+1)/factorial(k+2) for k in range(n))
            for i in range(-coeff):
                t=cut_off_n(t*term,n)
        elif coeff > 0:
            term = 1+sum(bernoulli_polynomial(1, k+1)*(from_monomial(mon))^(k+1)/factorial(k+1) for k in range(n))
            for i in range(coeff):
                t=cut_off_n(t*term,n)
        total=cut_off_n(total*t,n)
    return total

def cut_off_n(f,n):
    if f in ZZ:
        return f
    D=f.dict()
    for exp,coeff in f.dict().items():
        if sum(exp)>n:
            del D[exp]
    return coho_ring(D)

def restrict(f,n):
    D=f.dict()
    for exp,coeff in f.dict().items():
        if sum(exp)!=n:
            del D[exp]
    return coho_ring(D)

def specialize(f):
    if f in ZZ:
        return f
    return f(y=x^-1)

def determinant_fast(f,n):
    if f in ZZ:
        return 1
    else:
        total=1
        for coeff,mon in f:
            term=1+sum((coeff*from_monomial(mon))^(k+1)/factorial(k+1) for k in range(n+1))
            total=cut_off_n(total*term,n+1)
        return total

def determinant_quot_fast(Z,n, rank,N):
    det=1
    v_1=1
    v_2=1
    Q_2=1
    for i in range(rank):
        v_1=cut_off_n(v_1*(1+sum((w[i])^(k+1)/factorial(k+1) for k in range(n*N+1))),n*N+1)
    for i in range(n):
        v_2=cut_off_n(v_2*v_1,n*N+1)
    for i in range(len(Z)):
        Q = specialize(character(Z[i]))*(z[i])
        det = cut_off_n(det* determinant_fast(Q,n*N+1),n*N+1)
    for i in range(rank):
        Q_2=cut_off_n(Q_2*det,n*N+1)
    return cut_off_n(Q_2*v_2,n*N+1)

def determinant_fast_inv(f,n):
    if f in ZZ:
        return 1
    else:
        total=1
        for coeff,mon in f:
            term=1+sum((-coeff*from_monomial(mon))^(k+1)/factorial(k+1) for k in range(n+1))
            total=cut_off_n(total*term,n+1)
        return total

def determinant_quot_fast_inv(Z,n, rank,N):
    det=1
    v_1=1
    v_2=1
    Q_2=1
    for i in range(rank):
        v_1=cut_off_n(v_1*(1+sum((-w[i])^(k+1)/factorial(k+1) for k in range(n*N+1))),n*N+1)
    for i in range(n):
        v_2=cut_off_n(v_2*v_1,n*N+1)
    for i in range(len(Z)):
        Q = specialize(character(Z[i]))*(z[i])
        det = cut_off_n(det* determinant_fast_inv(Q,n*N+1),n*N+1)
    for i in range(rank):
        Q_2=cut_off_n(Q_2*det,n*N+1)
    return cut_off_n(Q_2*v_2,n*N+1)

def Verlinde_Num_quot_fast(n, r):
    total=0
    for Z in fixed_locus(n,1):
        add=top_chern(-Nvir(Z))*\
            restrict(todd_class_fast(Nvir(Z),n+1)*determinant_quot_fast(Z,n,r,1),n+1)
        total+=add
    return total(l1=0,w1=0,w2=0,w3=0,w4=0,w5=0,w6=0,w7=0,w8=0,w9=0)

def Verlinde_Num_quot_fast_inv(n,N, r):
    total=0
    for Z in fixed_locus(n,N):
        add=top_chern(-Nvir(Z))*\
            restrict(todd_class_fast(Nvir(Z),n*N+1)*determinant_quot_fast_inv(Z,n,r,N),n*N+1)
        total+=add
    return total(l1=0,l2=0,l3=0,l4=0,l5=0,w1=t*w1,w2=0,w3=0,w4=0,w5=0,w6=0,w7=0,w8=0,w9=0)


def deg0(f):
    if f==0: 
        return 0
    return degn(f(t=b*t,
                         l1=b*t*l1,l2=0,l3=0,l4=0,l5=0,
                         w1=b*t*w1,w2=b*t*w2,w3=b*t*w3,w4=0,w5=0,w6=0,w7=0,w8=0,w9=0),0)

def deg0n(f,n):
    if f==0: 
        return 0
    return degn(f(t=b*t,
                         l1=b*t*l1,l2=b*t*l2,l3=0,l4=0,l5=0,
                         w1=b*t*w1,w2=b*t*w2,w3=b*t*w3,w4=0,w5=0,w6=0,w7=0,w8=0,w9=0),n)

def deg1(f):
    if f==0: 
        return 0
    return degn(f(t=b*t,m=b*m,
                         l1=b*l1,l2=b*l2,l3=b*l3,l4=b*l4,l5=b*l5,
                         w1=b*w1,w2=b*w2,w3=b*w3,w4=b*w4,w5=b*w5),-1)

def deg02(f):
    if f==0: 
        return 0
    return pretty(degn(f(t=b*t,m=b*m,w=b*w,v=b*v),0))

def set_exp(f):
    for i in range(len(KT_vars)):
        f=SR(f).subs(KT_ring.gens()[i]==exp(coho_ring.gens()[i]))
    return f
def simplify_weight(f):
    return f(l1=0,l2=l1,l3=2*l1,l4=3*l1,l5=4*l5)

def symmetrize(f):
    if f in ZZ:
        return f
    return f(l1=w1,l2=w2,l3=w3,l4=w4,l5=w5,w1=l1,w2=l2,w3=l3,w4=l4,w5=l5)


def symmetrizeK(f):
    if f in ZZ:
        return f
    return f(z1=v1,z2=v2,z3=v3,z4=v4,z5=v5,v1=z1,v2=z2,v3=z3,v4=z4,v5=z5)

def all_check(start,finish):
    with open("V2d-red-checked.txt") as f:
        lines = f.read().splitlines()
    if len(lines)==0:
        n,N,r=[0,0,0]
    else:
        last_checked=lines[len(lines)-1]
        n,N,r=[int(a) for a in last_checked.split(",")]
    for i in range(start,finish+1):
        for j in range(1,4):
            for k in range(1,4):
                n,N,r=[i,j,k]
                print("checking n="+str(n)+", N="+str(N)+", r="+str(r))
                if str(n)+','+str(N)+','+str(r) in lines:
                    print("n="+str(n)+", N="+str(N)+", r="+str(r)+" already checked")
                else:
                    a=ver_compare(n,N,r)
                    V=deg0(set_exp(Verlinde_Num_quot(n,N,r))).coefficient(w1^2)
                    diff=a-V
                    if diff==0:
                        writeline("V2d-red-checked.txt",str(n)+','+str(N)+','+str(r))
                        print("checked")
                    else:
                        print("failed")
        print("all checked for: n="+str(i))
                        
                    
def writeline(file,s):
    g = open(file, "a")
    g.write(s+"\n")
    g.close()
                
                
def clear_all_files():
    open('V2d-red-checked.txt', 'w').close()
            
        
def binom(n,k):
    return prod((n-i) for i in range(k))/prod((k-i) for i in range(k))
    
    
def ver_compare(n,N,r):
    return -N/(2*(N+r)) *  binom((N+r)*n,r*n)

def ver10_compare(n,r):
    return ((n-1) *  binom(r+1,2)-1)/6

def c10_compare(n,r):
    return ((r-1)^2 +(n-2)*binom(r,2)-1)/6

def check01(start,finish):
    # check for n,N,r, up to r=5
    with open("V2d01-red-checked.txt") as f:
        lines = f.read().splitlines()
    for i in range(start,finish+1):
        for k in range(1,5):
            n,r=[i,k]
            print("checking n="+str(n)+", r="+str(r))
            if str(n)+','+str(r) in lines:
                print("n="+str(n)+", r="+str(r)+" already checked")
            else:
                a=ver10_compare(n,r)
                X=deg0(set_exp(Verlinde_Num_quot(n,1,r)))
                V1=X.coefficient(w1^2)
                V0=X(w1=0)
                diff=a-V0/V1
                if V1==0:
                    if V0==0:
                        writeline("V2d01-red-checked.txt",str(n)+','+str(r))
                        print("checked")
                        continue
                    else:
                        print("failed")
                        print(a)
                        print(V0)
                        print(V1)
                diff=a-V0/V1
                if diff==0:
                    writeline("V2d01-red-checked.txt",str(n)+','+str(r))
                    print("checked")
                else:
                    print("failed")
                    print(a)
                    print(V0)
                    print(V1)
        print("all checked for: n="+str(i))
        
def check01_inv(start,finish):
    with open("V2d01-red-checked.txt") as f:
        lines = f.read().splitlines()
    for i in range(start,finish+1):
        for k in range(1,5):
            n,r=[i,k]
            print("checking n="+str(n)+", r="+str(-r))
            if str(n)+','+str(-r) in lines:
                print("n="+str(n)+", r="+str(-r)+" already checked")
            else:
                a=ver10_compare(n,-r)
                X=deg0(set_exp(Verlinde_Num_quot_inv(n,1,r)))
                V1=X.coefficient(w1^2)
                V0=X(w1=0)
                if V1==0:
                    if V0==0:
                        writeline("V2d01-red-checked.txt",str(n)+','+str(-r))
                        print("checked")
                        continue
                    else:
                        print("failed")
                        print(a)
                        print(V0)
                        print(V1)
                diff=a-V0/V1
                if diff==0:
                    writeline("V2d01-red-checked.txt",str(n)+','+str(-r))
                    print("checked")
                else:
                    print("failed")
                    print(a)
                    print(V0)
                    print(V1)
        print("all checked for: n="+str(i))
        
def binom(n,k):
    if n==0 and k==0:
        return 1
    if n==0:
        return 0
    if n>0 and k<0:
        return 0
    if k>=0:
        return ((1+b)^n).taylor(b,0,k).list()[k]
    else:
        return ((1+b^-1)^n).taylor(b,0,-k).list()[-1*k]
    
def all_check_inv(start,finish):
    with open("V2d-red-checked_inv.txt") as f:
        lines = f.read().splitlines()
    for i in range(start,finish+1):
        for j in range(1,4):
            for k in range(1,4):
                n,N,r=[i,j,k]
                if r==N: 
                    continue
                print("checking n="+str(n)+", N="+str(N)+", r=-"+str(r))
                if str(n)+','+str(N)+',-'+str(r) in lines:
                    print("n="+str(n)+", N="+str(N)+", r=-"+str(r)+" already checked")
                else:
                    a=ver_compare(n,N,-r)
                    VV=deg0(set_exp(Verlinde_Num_quot_inv(n,N,r)))
                    if VV==0:
                        V=0
                    else:
                        V=VV.coefficient(w1^2)
                    diff=a-V
                    if diff==0:
                        writeline("V2d-red-checked_inv.txt",str(n)+','+str(N)+',-'+str(r))
                        print("checked")
                    else:
                        print("failed")
        print("all checked for: n="+str(i))
        
def check01_seg(start,finish):
    with open("s2d-checked.txt") as f:
        lines = f.read().splitlines()
    for i in range(start,finish+1):
        for k in range(2,5):
            n,r=[i,k]
            print("checking n="+str(n)+", r="+str(r))
            if str(n)+','+str(r) in lines:
                print("n="+str(n)+", r="+str(r)+" already checked")
            else:
                a=c10_compare(n,r)
                X=Chern_Num_quot(n,1,r)
                V1=X.coefficient(w1^2)
                V0=X(w1=0)
                diff=a-V0/V1
                if V1==0:
                    if V0==0:
                        writeline("s2d-checked.txt",str(n)+','+str(r))
                        print("checked")
                        continue
                    else:
                        print("faile1")
                        print(a)
                        print(V0)
                        print(V1)
                        break
                diff=a-V0/V1
                if diff==0:
                    writeline("s2d-checked.txt",str(n)+','+str(r))
                    print("checked")
                else:
                    print("failed")
                    print(a)
                    print(V0)
                    print(V1)
                    break

In [22]:
pretty(deg0n(Chern_Num_quot(2,2,3),0))

added1/5
added2/5
added3/5
added4/5
added5/5


-9/2*l1^2 - 6*l1*l2 - 9/2*l2^2 - 5*l1*w1 - 5*l2*w1 - w1^2 - 5*l1*w2 - 5*l2*w2 - 4*w1*w2 - w2^2 - 5*l1*w3 - 5*l2*w3 - 4*w1*w3 - 4*w2*w3 - w3^2

In [10]:
pretty(deg0n(set_exp(Verlinde_Num_quot(1,3,4)),0))

added1/3
added2/3
added3/3


-25*l1^2 - 15*l1*l2 - 25*l2^2 - 20*l1*w1 - 20*l2*w1 - 15/2*w1^2 - 20*l1*w2 - 20*l2*w2 - 15*w1*w2 - 15/2*w2^2 - 20*l1*w3 - 20*l2*w3 - 15*w1*w3 - 15*w2*w3 - 15/2*w3^2 + 5/4

In [27]:
pretty(deg0n(set_exp(Verlinde_Num_quot_inv(1,1,3)),0))

added1/1


-9/2*l1^2 - 3*l1*w1 - 1/2*w1^2 - 3*l1*w2 - w1*w2 - 1/2*w2^2 - 3*l1*w3 - w1*w3 - w2*w3 - 1/2*w3^2 + 1/12

In [28]:
pretty(deg0n(set_exp(Verlinde_Num_quot_inv(1,2,3)),0))

added1/2
added2/2


5/2*l1^2 + 4*l1*l2 + 5/2*l2^2 + 3*l1*w1 + 3*l2*w1 + w1^2 + 3*l1*w2 + 3*l2*w2 + 2*w1*w2 + w2^2 + 3*l1*w3 + 3*l2*w3 + 2*w1*w3 + 2*w2*w3 + w3^2 - 1/6

In [29]:
pretty(deg0n(set_exp(Verlinde_Num_quot_inv(1,3,3)),0))

added1/3
added2/3
added3/3


-1/2*l1^2 - l1*l2 - 1/2*l2^2 - l1*w1 - l2*w1 - 1/2*w1^2 - l1*w2 - l2*w2 - w1*w2 - 1/2*w2^2 - l1*w3 - l2*w3 - w1*w3 - w2*w3 - 1/2*w3^2 + 1/12

In [26]:
pretty(deg0n(set_exp(Verlinde_Num_quot_inv(1,4,3)),0))

added1/4
added2/4
added3/4
added4/4


0

In [28]:
pretty(deg0n(symmetrize(set_exp(Verlinde_Num_quot(2,3,2))),0))

added1/9
added2/9
added3/9
added4/9
added5/9
added6/9
added7/9
added8/9
added9/9


-63*l1^2 - 84*l1*w1 - 48*w1^2 - 12